In [1]:
import os
import sys
sys.path.append("..")
import torch
import subprocess
import traceback
import warnings
from pathlib import Path
from datetime import datetime
import pandas as pd
import numpy as np

from tqdm import tqdm
# n.pyimport hydra
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split
from torchvision import transforms

from src.metric import LWLRAP
from src.models import get_model
import src.configuration as C
import src.utils as utils
from src.metric import LWLRAP
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.callbacks import EarlyStopping
from pytorch_lightning.loggers import MLFlowLogger

/opt/anaconda3/lib/python3.7/site-packages/pywt/_utils.py:6: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Iterable


In [6]:
# config
config_filename = 'EfficientNetSED001.yaml'
config = utils.load_config(f"configs/{config_filename}")
device = "cuda"

In [7]:
# data
df, datadir = C.get_metadata(config)

In [8]:
df

,data_type,f_max,f_min,recording_id,songtype_id,species_id,t_max,t_min
0,tp,5531.25,2531.250,003bec244,1.0,14,45.1307,44.5440
1,tp,11283.40,7235.160,006ab765f,1.0,23,46.0452,39.9615
2,tp,3281.25,562.500,007f87ba2,1.0,12,42.2720,39.1360
3,tp,4565.04,1464.260,0099c367b,4.0,17,55.1996,51.4206
4,tp,10852.70,947.461,009b760e6,1.0,10,52.5293,50.0854
...,...,...,...,...,...,...,...,...
1211,tp,843.75,93.750,fe8d9ac40,1.0,13,54.0960,53.4720
1212,tp,4031.25,2531.250,fea6b438a,1.0,4,45.7653,43.5787
1213,tp,8250.00,5906.250,ff2eb9ce5,1.0,0,16.0213,15.2267
1214,tp,10406.20,4781.250,ffb8d8391,1.0,5,16.6987,14.3467


In [50]:
config["loader"]["valid"]["batch_size"] = 20

In [51]:
loaders = {
    phase: C.get_loader(df_, datadir, config, phase)
    for df_, phase in zip([df], ["valid"])
}

In [77]:
df0 = df[["recording_id"]].copy()  # 0~10s
df0["t_min"] = 0
df0["t_max"] = 10

df1 = df[["recording_id"]].copy()  # 10~20s
df1["t_min"] = 10
df1["t_max"] = 20

df2 = df[["recording_id"]].copy()  # 20~30s
df2["t_min"] = 20
df2["t_max"] = 30

df3 = df[["recording_id"]].copy()  # 30~40s
df3["t_min"] = 30
df3["t_max"] = 40

df4 = df[["recording_id"]].copy()  # 40~50s
df4["t_min"] = 40
df4["t_max"] = 50

df5 = df[["recording_id"]].copy()  # 50~60s
df5["t_min"] = 50
df5["t_max"] = 60

pseudo_label_df = pd.concat([df0, df1, df2, df3, df4, df5]).reset_index(drop=True)  # 6つ繋げる(60sを10sごとの音声に6分割するので)
pseudo_label_df

,recording_id,t_min,t_max
0,003bec244,0,10
1,006ab765f,0,10
2,007f87ba2,0,10
3,0099c367b,0,10
4,009b760e6,0,10
...,...,...,...
7291,fe8d9ac40,50,60
7292,fea6b438a,50,60
7293,ff2eb9ce5,50,60
7294,ffb8d8391,50,60


In [78]:
pseudo_label_df["t_max"].value_counts()

50    1216
30    1216
10    1216
60    1216
40    1216
20    1216
Name: t_max, dtype: int64

In [79]:
for fold in range(5):
    # model
    print(f"fold-{fold}")
    model = get_model(config)
    model_name = "EfficientNetSED"
    output_dir = Path("output/0119_184426")
    try:
        ckpt = torch.load(output_dir / f'{model_name}-{fold}-v0.ckpt')  # TODO foldごとのモデルを取得できるようにする
    except:
        ckpt = torch.load(output_dir / f'{model_name}-{fold}.ckpt')  # TODO foldごとのモデルを取得できるようにする
    model.load_state_dict(ckpt['state_dict'])
    model.eval().to(device)

    preds = []
    output_key = "logit"
    with torch.no_grad():
        # xは複数のlist
        for x_list, y in tqdm(loaders['valid']):
            batch_size = x_list.shape[0]
            x = x_list.view(-1, x_list.shape[2], x_list.shape[3], x_list.shape[4])  # batch>1でも可
            x = x.to(device)
            output = model.model(x)
            output = output[output_key]
            output = output.view(batch_size, -1, 24)  # (20, 6, 24)
            pred = torch.argsort(output, dim=-1, descending=True)  
            pred = torch.argmin(pred, axis=2)  # (20, 6)
            pred = pred.detach().cpu().numpy() 
            preds.append(pred)

    pseudo_label_df[f"fold{fold}"] = np.vstack(preds).flatten()  # (n, 6) 

fold-0
Loaded pretrained weights for efficientnet-b2
100%|██████████| 61/61 [00:23<00:00,  2.55it/s]
fold-1
Loaded pretrained weights for efficientnet-b2
100%|██████████| 61/61 [00:24<00:00,  2.52it/s]
fold-2
Loaded pretrained weights for efficientnet-b2
100%|██████████| 61/61 [00:24<00:00,  2.50it/s]
fold-3
Loaded pretrained weights for efficientnet-b2
100%|██████████| 61/61 [00:24<00:00,  2.53it/s]
fold-4
Loaded pretrained weights for efficientnet-b2
100%|██████████| 61/61 [00:24<00:00,  2.51it/s]


In [80]:
pseudo_label_df

,recording_id,t_min,t_max,fold0,fold1,fold2,fold3,fold4
0,003bec244,0,10,13,3,14,8,13
1,006ab765f,0,10,10,13,16,8,10
2,007f87ba2,0,10,15,14,18,15,18
3,0099c367b,0,10,12,8,20,11,10
4,009b760e6,0,10,10,8,15,5,14
...,...,...,...,...,...,...,...,...
7291,fe8d9ac40,50,60,17,7,8,3,7
7292,fea6b438a,50,60,14,9,4,3,4
7293,ff2eb9ce5,50,60,5,3,4,4,4
7294,ffb8d8391,50,60,9,4,1,4,4


In [91]:
pseudo_label_list = [] 
for idx, row in pseudo_label_df.iterrows():
    recording_id = row
    try:
        pseudo_label = row["fold0":"fold4"].value_counts()[(row["fold0":"fold4"].value_counts() > 2)].values[0]
    except:
        pseudo_label = np.nan
    pseudo_label_list.append(pseudo_label)


In [92]:
pseudo_label_df["species_id"] = pseudo_label_list
pseudo_label_df

,recording_id,t_min,t_max,fold0,fold1,fold2,fold3,fold4,species_id
0,003bec244,0,10,13,3,14,8,13,NaN
1,006ab765f,0,10,10,13,16,8,10,NaN
2,007f87ba2,0,10,15,14,18,15,18,NaN
3,0099c367b,0,10,12,8,20,11,10,NaN
4,009b760e6,0,10,10,8,15,5,14,NaN
...,...,...,...,...,...,...,...,...,...
7291,fe8d9ac40,50,60,17,7,8,3,7,NaN
7292,fea6b438a,50,60,14,9,4,3,4,NaN
7293,ff2eb9ce5,50,60,5,3,4,4,4,3.0
7294,ffb8d8391,50,60,9,4,1,4,4,3.0


In [93]:
pseudo_label_df.species_id.value_counts()

3.0    434
5.0    193
4.0    140
Name: species_id, dtype: int64

In [94]:
pseudo_label_df = pseudo_label_df.dropna()

In [95]:
pseudo_label_df = pseudo_label_df[["recording_id", "species_id", "t_min", "t_max"]]

In [96]:
pseudo_label_df["species_id"] = pseudo_label_df["species_id"].astype(int)

In [97]:
pseudo_label_df

,recording_id,species_id,t_min,t_max
16,0297d886e,3,0,10
36,068f1b8e2,5,0,10
37,06a0c48c9,4,0,10
38,06c44d203,5,0,10
39,073e4d908,3,0,10
...,...,...,...,...
7281,fbd5bbb6e,5,50,60
7282,fc47cfba3,3,50,60
7283,fc6609050,3,50,60
7293,ff2eb9ce5,3,50,60


In [98]:
pseudo_label_df.to_csv("./input/rfcx-species-audio-detection/pseudo.csv", index=False)